In [1]:
import pygame
import sys
import numpy as np
import random
import pandas as pd
import time

pygame 2.1.2 (SDL 2.0.18, Python 3.7.6)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
width=12
height=12
pixel_size=(50,50)
display_size=(pixel_size[0]*width,pixel_size[1]*height)
record=[]
game_num=1

In [3]:
color_dict={
    "wall":(177,177,177),
    "target":(0,177,0),
    "normal":(255,255,255),
    "start":(177,0,0)
}

map_dict={
    "target":(7,2),
    "start":(2,height-1),
    "tank1":(7,3),
    "tank2":(6,2),
    "tank3":(random.randint(3,width-3),random.randint(2,height-3)),
    "tank4":(random.randint(2,width-2),random.randint(1,height-3)),
    "tank5":(random.randint(2,width-2),random.randint(1,height-3)),
    "tank6":(random.randint(2,width-2),random.randint(1,height-3)),
   }

wall_location=[]
for w in range(width):
    wall_location.append((w,0))
    wall_location.append((w,height-1))
    
for h in range(height):    
    wall_location.append((0,h))
    wall_location.append((width-1,h))
    
wall_location.remove(map_dict.get("start"))
wall_location.append((map_dict.get("start")[0],map_dict.get("start")[1]+1))
map_dict['wall']=wall_location

In [4]:
def draw_map():
    for i in range(width*height):
        w=i-width*(i//width)
        h=i//width
        
        if (w,h)==map_dict.get("target"):
            color=color_dict.get("target")
        elif (w,h)==map_dict.get("start"):
            color=color_dict.get("start")
        elif (w,h) in map_dict.get("wall"):
            color=color_dict.get("wall")
        else:
            color=color_dict.get("normal")
        
        pygame.draw.rect(game_map,color,(w*50,h*50,pixel_size[0],pixel_size[1]))

def reset():
    global person_site
    global game_state
    person_site=[map_dict.get("start")[0],map_dict.get("start")[1]]
    person.rect=(person_site[0]*pixel_size[0],person_site[1]*pixel_size[1],50,50)
    game_state=0
    reward.append(0)


In [5]:
class tank(pygame.sprite.Sprite):
    def __init__(self,filename,location):
        pygame.sprite.Sprite.__init__(self)
        self.image = pygame.image.load(filename).convert()
        self.image = pygame.transform.scale(self.image,(50,50))

        self.rect = self.image.get_rect()
        self.rect.topleft=(location[0]*pixel_size[0],location[1]*pixel_size[1])


In [6]:
def crash_detect(sprite1,sprite2):
    global game_num
    global game_state
    result=pygame.sprite.collide_mask(sprite1,sprite2)
    
    if result:
        if sprite2 in tank1_group:
            print("这是第{}局游戏，我死了".format(game_num))
            if game_num>20:
                print('最后20盘的胜率为{}'.format(record[-20:].count(1)/20))
            record.append(0)
            reward.append(-1)

        elif sprite2 in princess_group:
            print("这是第{}局游戏，我赢了".format(game_num))
            if game_num>20:
                print('最后20盘的胜率为{}'.format(record[-20:].count(1)/20))
            record.append(1)
            reward.append(1)


        game_num+=1
        game_state=1



In [7]:
def crash_wall():
    if (person_site[0],person_site[1]) in map_dict.get("wall"):
        return True
    else:
        return False

In [8]:
def move(direction):
    global person_site
    if direction=='up':
        person_site[1]-=1
        a= crash_wall()
        if a==True:
            person_site[1]+=1
        
    if direction=='down':
        person_site[1]+=1
        a= crash_wall()
        if a==True:
            person_site[1]-=1
    if direction=='left':
        person_site[0]-=1
        a= crash_wall()
        if a==True:
            person_site[0]+=1
    if direction=='right':
        person_site[0]+=1
        a= crash_wall()
        if a==True:
            person_site[0]-=1
        
    person.rect=(person_site[0]*pixel_size[0],person_site[1]*pixel_size[1],50,50)
    crash_detect(person,tank1)       
    crash_detect(person,tank2)   
    crash_detect(person,tank3)       
    crash_detect(person,tank4) 
    crash_detect(person,tank5)       
    crash_detect(person,tank6)   
    crash_detect(person,princess)         



In [9]:
qtable=pd.DataFrame(np.zeros(((width-2)*(height-2)+1,4)),columns=['up','down','left','right'])

In [10]:
s_index={}
for i in range(width-2):
    for j in range(height-2):
        s_index.update({(i+1,j+1):(i+(j*(width-2)))})

s_index.update({map_dict.get("start"):(width-2)*(height-2)})   


In [11]:
epsilon=0.8
reward=[0]
gamma=0.9
lr=0.1

In [12]:
def pick_a(s):
    index=s_index.get(s)
    actions=qtable.iloc[index]
    if random.random()>epsilon:
        action=actions.sample().index[0]
    else:
        x=actions.sample(4)
        action= x.index[x.argmax()]
    return action


def one_turn():
    s=(int(person_site[0]),int(person_site[1]))
    a=pick_a(s)
    move(a)
    s_next=(int(person_site[0]),int(person_site[1]))
    update_qtable(s,a,s_next)
    if game_state==1:           
        reset()

        
def update_qtable(s,a,s_next):
    q_sa=qtable.iloc[s_index.get(s)].get(a)    
    q_sa_next=np.max(qtable.iloc[s_index.get(s_next)])
    q_sa+=lr*(gamma*q_sa_next+reward[-1]-q_sa)
    qtable.iloc[s_index.get(s)][a]=q_sa
    

In [13]:
pygame.init()
clock = pygame.time.Clock()
pygame.display.set_caption("AI训练游戏")

screen = pygame.display.set_mode(display_size)
game_map=pygame.surface.Surface(display_size)
draw_map()

tank1_group = pygame.sprite.Group()
tank2_group = pygame.sprite.Group()
princess_group = pygame.sprite.Group()
person_group=pygame.sprite.Group()

tank1 = tank("img/tank.jpg",map_dict.get("tank1"))
tank2 = tank("img/tank.jpg",map_dict.get("tank2"))
tank3 = tank("img/tank.jpg",map_dict.get("tank3"))
tank4 = tank("img/tank.jpg",map_dict.get("tank4"))
tank5 = tank("img/tank.jpg",map_dict.get("tank5"))
tank6 = tank("img/tank.jpg",map_dict.get("tank6"))
tank1_group.add(tank1)
tank1_group.add(tank2)
tank1_group.add(tank3)
tank1_group.add(tank4)
tank1_group.add(tank5)
tank1_group.add(tank6)

princess=tank('img/sidalin.jpeg',map_dict.get("target"))
princess_group.add(princess)

person_site=[map_dict.get("start")[0],map_dict.get("start")[1]]
person=tank('img/maintank.jpeg',person_site)
person_group.add(person)

reset()

while True:
    clock.tick(60)

    screen.blit(game_map,(0,0))
    tank1_group.draw(screen)
    princess_group.draw(screen)
    screen.blit(person.image,person.rect)
    tank2_group.draw(screen)
    
    
    one_turn()
    if game_num<50:
        pass
    elif game_num<100:
        epsilon=0.9
    elif game_num>100:    
        epsilon=0.95
        

        
    for event in pygame.event.get():
        if event.type ==pygame.QUIT:
            pygame.quit()
            sys.exit()
            

        
#       if event.type ==pygame.KEYDOWN:
#           if event.key == pygame.K_UP:
#               move('up')
#           if event.key == pygame.K_DOWN:
#               move('down')
#           if event.key == pygame.K_LEFT:
#               move('left')
#           if event.key == pygame.K_RIGHT:
#               move('right')
#           if event.key == pygame.K_SPACE:
                
                
     
    pygame.display.update()

这是第1局游戏，我死了
这是第2局游戏，我死了
这是第3局游戏，我死了
这是第4局游戏，我死了
这是第5局游戏，我死了
这是第6局游戏，我死了
这是第7局游戏，我死了
这是第8局游戏，我死了
这是第9局游戏，我死了
这是第10局游戏，我死了
这是第11局游戏，我死了
这是第12局游戏，我死了
这是第13局游戏，我死了
这是第14局游戏，我死了
这是第15局游戏，我死了
这是第16局游戏，我死了
这是第17局游戏，我死了
这是第18局游戏，我死了
这是第19局游戏，我死了
这是第20局游戏，我死了
这是第21局游戏，我死了
最后20盘的胜率为0.0
这是第22局游戏，我死了
最后20盘的胜率为0.0
这是第23局游戏，我死了
最后20盘的胜率为0.0
这是第24局游戏，我死了
最后20盘的胜率为0.0
这是第25局游戏，我死了
最后20盘的胜率为0.0
这是第26局游戏，我死了
最后20盘的胜率为0.0
这是第27局游戏，我死了
最后20盘的胜率为0.0
这是第28局游戏，我死了
最后20盘的胜率为0.0
这是第29局游戏，我死了
最后20盘的胜率为0.0
这是第30局游戏，我死了
最后20盘的胜率为0.0
这是第31局游戏，我死了
最后20盘的胜率为0.0
这是第32局游戏，我死了
最后20盘的胜率为0.0
这是第33局游戏，我死了
最后20盘的胜率为0.0
这是第34局游戏，我赢了
最后20盘的胜率为0.0
这是第35局游戏，我死了
最后20盘的胜率为0.05
这是第36局游戏，我死了
最后20盘的胜率为0.05
这是第37局游戏，我死了
最后20盘的胜率为0.05
这是第38局游戏，我死了
最后20盘的胜率为0.05
这是第39局游戏，我死了
最后20盘的胜率为0.05
这是第40局游戏，我死了
最后20盘的胜率为0.05
这是第41局游戏，我死了
最后20盘的胜率为0.05
这是第42局游戏，我死了
最后20盘的胜率为0.05
这是第43局游戏，我死了
最后20盘的胜率为0.05
这是第44局游戏，我死了
最后20盘的胜率为0.05
这是第45局游戏，我死了
最后20盘的胜率为0.05
这是第46局游戏，我死了
最后20盘的胜率为0.05
这是第47局游戏，我赢了
最后20盘的胜率为0.05
这是第48局游戏，我死了
最后20盘的胜率为0.1
这是第49局游戏

SystemExit: 

d:\miniconda3\lib\site-packages\IPython\core\interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
